In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image
import scipy.sparse as sp
import scipy
import time
import itertools
import scipy
import cvxpy as cp
import random
import matplotlib
import math
import gurobipy as gp
from gurobipy import GRB

def plot(data):
    figure, axis = plt.subplots(2, 3)

    axis[0][0].plot(data.cost)
    axis[0][0].set_title("cost transport")
    #axis[0].set_ylim((0, 2))

    axis[0][1].plot(data.time_child)
    axis[0][1].set_title("time to find a child")
    #axis[1].set_ylim((0, 0.01))

    axis[0][2].plot(data.time_model)
    axis[0][2].set_title("LP time")
    #axis[2].set_ylim((0, 0.5))

    axis[1][0].plot(data.n_children)
    axis[1][0].set_title("number sites")

    axis[1][1].boxplot(data.memory)
    axis[1][1].set_title("number of memory slot")

    axis[1][2].plot(np.array(data.cost[:-1])-np.array(data.cost[1:]))
    axis[1][2].set_title("improvements")


    figure.set_size_inches(18.5, 10.5)

    plt.show()

    print(f"the final cost is {data.cost[-1]}")
    
def stats(data):
    print("obj_cost: ", data.cost[-1])
    print("average_memory: ", np.mean(data.memory))
    print("average_time_LP: ", np.mean(data.time_model))
    print("average_children_sampled: ", np.mean(data.children_sampled))

def create_array(orig_array, percentage):
    num_rows = max(1, int(orig_array.shape[0] * percentage))
    unique_indices = np.random.choice(orig_array.shape[0], size=num_rows, replace=False)
    unique_rows = orig_array[unique_indices, :]
    num_repeats = int(np.ceil(orig_array.shape[0] / float(num_rows)))
    new_array = np.repeat(unique_rows, num_repeats, axis=0)[:orig_array.shape[0], :]
    return new_array

class GeneticAlgorithm:
    def __init__(self, img_list):
        self.initialize_stats()
        self.initialize_data(img_list)
        self.timed_initialization(self.initialize_omega, "omega")
        self.timed_initialization(self.initialize_cost_vector, "cost vector")
        self.timed_initialization(self.initialize_model, "full model")
        
    def initialize_stats(self):
        self.cost, self.time_child, self.children_sampled = [], [], []
        self.time_model, self.len_children, self.n_children, self.memory = [], [], [], []

    def initialize_data(self, img_list):
        self.N, self.dim = img_list[0].shape[0], len(img_list[0].shape)
        self.imgs = np.array([img.flatten() for img in img_list])
        self.b_eq = np.concatenate(self.imgs)
        self.non_zero_indices = []
        self.n_imgs = len(self.imgs)

    def timed_initialization(self, method, message):
        start_time = time.time()
        print(f"Initializing {message}...")
        method()
        print(f"{message.capitalize()} initialized in {time.time() - start_time:.3f} s")

    def initialize_omega(self):
        #the north west rule is used to initialise omega
        #we start with the first pixel of each image
        indices = np.zeros(self.n_imgs)
        omega = np.array(indices.copy())
        b = np.array([img[0] for img in self.imgs])

        current_gamma = [np.min(b)]
        while np.min(indices) < self.N**self.dim-1:
            gamma = np.min(b)
            b-=gamma
            low = np.where(b < 1e-9)[0]
            indices[low] +=1
            indices = np.clip(indices, a_min=0, a_max=self.N**self.dim-1)
            b[low] = self.imgs[low,indices[low].astype('int')]
            
            omega = np.vstack((omega,indices.copy()))
            current_gamma.append(gamma)
        
        self.current_gamma = np.array(current_gamma)
        self.active_indices = np.array(omega).astype('int')

    def initialize_cost_vector(self):
        self.current_cost_vector = self.get_cost(self.active_indices)

    def initialize_model(self):
        start = time.time()

        # Create indices for the sparse constraint matrix
        indices_row = np.array([])
        indices_col = np.array([])
        for i in range(self.n_imgs):
            for indices in range(self.N ** self.dim):
                gamma_indices = np.where(self.active_indices.transpose()[i] == indices)[0]
                indices_row = np.concatenate((indices_row, gamma_indices))
                indices_col = np.concatenate((indices_col, np.ones(len(gamma_indices)) * (indices + i * (self.N ** self.dim))))

        # Create the constraint matrix A_eq as a sparse matrix
        A_eq = sp.csr_matrix((np.ones(len(indices_col)), (indices_col, indices_row)),
                             shape=((self.N ** self.dim) * self.n_imgs, len(self.active_indices)))

        # Initialize the Gurobi model
        self.m = gp.Model("model")

        # Add variables to the model
        self.gamma = self.m.addMVar(shape=len(self.current_cost_vector), vtype=GRB.CONTINUOUS, name="gamma")

        # Set the objective function
        self.m.setObjective(self.current_cost_vector @ self.gamma, GRB.MINIMIZE)

        # Set Gurobi parameters
        self.m.Params.OutputFlag = False
        self.m.Params.Method = 1
        self.m.Params.FeasibilityTol = 1e-8

        # Add constraints to the model
        self.m.addConstr(A_eq @ self.gamma == self.b_eq, name="eq_c")

        # Enable warm starts
        self.m.Params.LPWarmStart = 2
        self.gamma.PStart = self.current_gamma

        # Optimize the model
        self.m.optimize()

        # Store the primal and dual solutions
        self.current_gamma = np.array(self.gamma.X)
        self.current_kantorovich = np.array(self.m.PI)

        # Store the time and cost
        self.time_model.append(time.time() - start)
        self.cost.append(self.m.ObjVal)

        # Store the constraint matrix for future use
        self.constraints_RMP = A_eq

    def barycentric_distance(self, indices_list):
        barycenter = np.mean(indices_list / self.N, axis=0)
        return np.mean(np.sum((x - barycenter) ** 2, axis=0) for x in indices_list / self.N)

    def get_cost(self, vector):
        indices_list = [
            np.array(np.unravel_index(vector.transpose()[i], [self.N] * self.dim))
            for i in range(self.n_imgs)
        ]
        return self.barycentric_distance(np.array(indices_list))

    def compute_gain(self, cost, children):
        kantorovich_terms = [
            self.current_kantorovich[i * (self.N ** self.dim):(i + 1) * (self.N ** self.dim)][children.transpose()[i]]
            for i in range(self.n_imgs)
        ]
        return np.sum(kantorovich_terms, axis=0) - cost

    def find_best_child(self):
        parent = create_array(self.non_zero_indices.copy().transpose(), self.parent_to_children)
        index = random.sample(range(0, self.n_imgs), self.parent_changed)
        parent[index] = np.random.randint(0, self.N ** self.dim, size=len(parent[1]))
        children = parent.copy().transpose()
        gain = self.compute_gain(self.get_cost(children), children)
        best_children = children[gain > 0]
        
        if not self.random_selection:
            best_children = children[np.argsort(gain)[-int(len(best_children) * self.turnover_rate):]]
        else:
            best_children = np.random.choice(best_children, size=int(len(best_children) * self.turnover_rate), replace=False)

        if self.add_neighbours:
            best_children = self.add_neighbouring_children(best_children, index)
        
        return best_children

    def add_neighbouring_children(self, best_children, index):
        for i in self.neighbours:
            parent[index] = np.clip(self.non_zero_indices.copy().transpose()[index] + i, 0, self.N ** 2 - 1)
            children = parent.copy().transpose()
            gain = self.compute_gain(self.get_cost(children), children)
            additional_children = children[gain > 0]
            best_children = np.vstack((best_children, additional_children))
        return best_children


    def set_run_parameters(self, max_iter, max_sample, max_runtime, beta, turnover_rate,
                           parent_to_children, random_selection, parent_changed, add_neighbours, radius):
        self.beta = beta
        self.turnover_rate = turnover_rate
        self.parent_to_children = parent_to_children
        self.random_selection = random_selection
        self.parent_changed = parent_changed
        self.add_neighbours = add_neighbours
        self.radius=radius
        self.neighbours= []
        for i in range(int(-self.N**2/2),int(self.N**2/2)):
            xx = np.sign(i)*i % self.N*np.sign(i)
            yy = int(i / self.N)
            mask = (xx)**2 + (yy)**2 < self.radius**2
            if not mask or i==0: continue
            self.neighbours.append(i)
        
        
    def prune_and_reinitialize_model(self):
        if self.beta>3:
            remove_value = self.beta-1
        else:
            remove_value = 1
        zero_indices = np.where(self.current_gamma == 0)[0][:int(remove_value*(self.N**self.dim)*self.n_imgs)]
        self.active_indices = np.delete(self.active_indices, zero_indices, axis=0)
        self.current_cost_vector = np.delete(self.current_cost_vector, zero_indices)
        self.current_gamma = np.delete(self.current_gamma, zero_indices)
        self.initialize_model()
                

    def get_mean(self, par):
        indices = np.array([[np.unravel_index(self.active_indices.transpose()[i], (self.N, self.N))[j] for i in range(self.n_imgs)] for j in range(2)]) 
        indices = indices.transpose((1,0,2))
        mean = [np.sum([par[i]*indices[i][j] for i in range(self.n_imgs)], axis=0).astype('int') for j in range(2)]        
        mean = np.ravel_multi_index(mean, (self.N, self.N))
        gamma = sp.csr_matrix((self.current_gamma, (self.active_indices.transpose()[0], mean)),
                              shape=(self.N ** 2, self.N ** 2))
        return 1-gamma.todense().transpose().dot(self.imgs[0]).reshape(self.N, self.N)

    
    def update_model_with_children(self, best_children):
        # Update active indices and gamma
        self.active_indices = np.vstack((self.active_indices, best_children))
        self.current_gamma = np.append(self.current_gamma, np.zeros(best_children.shape[0]))
        
        # Update cost vector
        new_cost = self.get_cost(best_children)
        self.current_cost_vector = np.append(self.current_cost_vector, new_cost)
        
        # Add new variables to the model
        new_vars = self.m.addMVar(len(new_cost), vtype=GRB.CONTINUOUS, name="new_gamma")
        self.m.setObjective(self.current_cost_vector @ np.concatenate([self.gamma.X, new_vars.X]), GRB.MINIMIZE)
        
        # Update constraints
        idx = best_children.copy().transpose()
        shift = np.array([i * self.N ** self.dim for i in range(self.n_imgs)]).reshape(-1, 1)
        idx = (idx + shift).transpose()
        reshaped_arr = idx.reshape(-1)

        second_elements = np.repeat(np.arange(len(best_children)), self.n_imgs)
        stacked_arr = np.vstack((reshaped_arr, second_elements))
        
        new_A = sp.csr_matrix((np.ones(len(best_children) * self.n_imgs), (stacked_arr[0], stacked_arr[1])),
                              shape=((self.N ** self.dim) * self.n_imgs, len(best_children)))

        new_constr = self.m.addConstr(self.constraints_RMP @ new_vars == np.zeros(len(best_children) * self.n_imgs),
                                      name="new_eq_c")

        # Update the constraints matrix
        self.constraints_RMP = sp.hstack((self.constraints_RMP, new_A))

        # Update the warm start
        new_vars.PStart = np.zeros(best_children.shape[0])
        self.gamma.PStart = self.current_gamma

    def solve_model(self):
        # Just reoptimize the existing model
        self.m.optimize()
        
        # Update solutions
        self.current_gamma = np.array(self.gamma.X)
        self.current_kantorovich = np.array(self.m.PI)
        self.cost.append(self.m.ObjVal)
    
    def run(self, max_iter, max_sample, max_runtime, beta, turnover_rate,
            parent_to_children, random_selection, parent_changed, add_neighbours, radius):
        
        self.set_run_parameters(max_iter, max_sample, max_runtime, beta, turnover_rate,
                                parent_to_children, random_selection, parent_changed, add_neighbours, radius)
        
        start_runtime = time.time()
        n_memory = 0

        for _ in tqdm(range(self.max_iter)):
            if time.time() - start_runtime > self.max_runtime:
                break

            start_iter = time.time()
            self.non_zero_indices = self.active_indices[np.nonzero(self.current_gamma)]
            best_children = self.find_best_child()
            
            self.update_model_with_children(best_children)
            self.time_child.append(time.time() - start_iter)
            n_memory += 1

            if self.active_indices.shape[0] > int(self.beta * (self.N ** self.dim) * self.n_imgs):
                self.memory.append(n_memory)
                n_memory = 0
                self.prune_and_reinitialize_model()
                continue

            self.solve_model()
        
    def plot(self):
        if self.n_imgs==2:
            plt.close()
            fig, axs = plt.subplots(2, 6)
            axs[0][0].imshow(self.get_mean((1,0)), cmap='gray')
            axs[0][1].imshow(self.get_mean((0.95,0.05)), cmap='gray')
            axs[0][2].imshow(self.get_mean((0.9,0.1)), cmap='gray')
            axs[0][3].imshow(self.get_mean((0.8,0.2)), cmap='gray')
            axs[0][4].imshow(self.get_mean((0.7,0.3)), cmap='gray')
            axs[0][5].imshow(self.get_mean((0.6,0.4)), cmap='gray')
            axs[1][0].imshow(self.get_mean((0.5,0.5)), cmap='gray')
            axs[1][1].imshow(self.get_mean((0.4,0.6)), cmap='gray')
            axs[1][2].imshow(self.get_mean((0.3,0.7)), cmap='gray')
            axs[1][3].imshow(self.get_mean((0.2,0.8)), cmap='gray')
            axs[1][4].imshow(self.get_mean((0.1,0.9)), cmap='gray')
            axs[1][5].imshow(self.get_mean((0,1)), cmap='gray')
            plt.show()
        if self.n_imgs==3:
            plt.close()
            fig, axs = plt.subplots(4, 4)
            axs[0][0].imshow(self.get_mean((1,0,0)), cmap='gray')
            axs[0][1].imshow(self.get_mean((0.67,0.33,0)), cmap='gray')
            axs[0][2].imshow(self.get_mean((0.37,0.63,0)), cmap='gray')
            axs[0][3].imshow(self.get_mean((0,1,0)), cmap='gray')
            axs[1][0].imshow(self.get_mean((0.67,0,0.33)), cmap='gray')
            axs[1][1].imshow(self.get_mean((0.5,0.25,0.25)), cmap='gray')
            axs[1][2].imshow(self.get_mean((0.25,0.5,0.25)), cmap='gray')
            axs[2][0].imshow(self.get_mean((0.33,0,0.67)), cmap='gray')
            axs[2][1].imshow(self.get_mean((0.25,0.25,0.5)), cmap='gray')
            axs[3][0].imshow(self.get_mean((0,0,1)), cmap='gray')  
            plt.show()
        if self.n_imgs>3:
            plt.close()
            fig, axs =plt.subplots(1,self.n_imgs+1)
            for i in range(self.n_imgs):
                axs[i].imshow(1-self.imgs[i].reshape(self.N,self.N), cmap='gray')
            axs[self.n_imgs].imshow(self.get_mean(tuple([1/self.n_imgs])*self.n_imgs), cmap='gray')
            plt.show()


ModuleNotFoundError: No module named 'cvxpy'

In [ ]:

path_img1 = "dolphin_64.jpg"
path_img2 = "bird_64.jpg"
path_img3 = "star_64.jpg"
path_img4 = "flower_64.jpg"

dim=64 
n_imgs = 2

#paths = [f"L/L{i}.jpg" for i in range(1,8)]
paths = [path_img1,path_img2,path_img3]
img_list = [np.array(Image.open(path).convert('L')) for path in paths]
#img_list = [np.random.random((dim,dim)) for i in range(n_imgs)]
img_list = [1-img/255 for img in img_list]
img_list = [(img/np.sum(img))*img.shape[0] for img in img_list]

In [ ]:
ga = Genetic_Algorithm(img_list)

In [ ]:
turnover_rate = 1 
beta = 4 
parent_to_children = 1 
random_selection = False 
parent_changed = 1 
add_neighbours = True
radius = 2

In [ ]:
ga.run(max_iter=2000, max_sample=30, max_runtime=60, beta=beta, turnover_rate=turnover_rate, parent_to_children=parent_to_children, random_selection=random_selection, parent_changed=parent_changed, add_neighbours=add_neighbours, radius=radius)